In [1]:
import pandas

import pygsheets
import numpy
import scipy
import re

import matplotlib
import matplotlib.pyplot as mp
import matplotlib.cm

import textwrap
import seaborn

In [2]:
pandas.__version__

'1.3.5'

# import data directly from google sheets

In [167]:
credentials_directory = '/Users/baird/Dropbox/_google-api/'
gc = pygsheets.authorize(client_secret=credentials_directory+'client_secret.json')
spreadsheet = gc.open_by_key('1BshhQvcBOhqW4OPkMHXkZBdz9zYvS9q4ockW9yKufGQ') # specific doc for report
#spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek') # original

#spreadsheet[1] "Gas Pipelines" tab is the second index
gas_pipes = spreadsheet.worksheet('title','Gas pipelines').get_as_df(start='A2')
oil_pipes = spreadsheet.worksheet('title', 'Oil/NGL pipelines').get_as_df(start='A2')

pipes_df_orig = pandas.concat([oil_pipes, gas_pipes], ignore_index=True)

#get country ratios sheet
country_ratios_df = spreadsheet.worksheet('title', 'Country ratios by pipeline').get_as_df()

In [168]:
owners_df_orig = spreadsheet.worksheet('title', 'Pipeline operators/owners (1/3)').get_as_df(start='A2')
owners_df_orig = owners_df_orig.loc[owners_df_orig.ProjectID!='']
owners_df_orig = owners_df_orig.loc[owners_df_orig.Wiki!='']
owners_df = owners_df_orig.replace('',numpy.nan)

owner_parent_links_df = spreadsheet.worksheet('title', 'Owner–parent relationships (2/3)').get_as_df(start='A2')
# only keep the owners with a checked relationship
owner_parent_links_df = owner_parent_links_df.loc[owner_parent_links_df['Parent–Owner Relationship Checked?']=='yes']
owner_parent_links_df.replace('',numpy.nan,inplace=True)

parents_df = spreadsheet.worksheet('title', 'Parent metadata (3/3)').get_as_df(start='A2')
parents_df = parents_df.loc[parents_df.Parent!='']

owners_df.set_index('ProjectID', inplace=True)
owner_parent_links_df.set_index('Owner', inplace=True)
parents_df.set_index('Parent', inplace=True)

# ****************************************

## create list of owner and parent column names
owner_pct_col_names = []
owner_col_names = []

parent_pct_col_names = []
parent_col_names = []

for num in range(1,11+1):
    owner_pct_col = f'Owner{num}%'
    owner_pct_col_names.append(owner_pct_col)
    
    owner_col = f'Owner{num}'
    owner_col_names.append(owner_col)
    
    parent_pct_col = f'Parent{num}%'
    parent_pct_col_names.append(parent_pct_col)
    
    parent_col = f'Parent{num}'
    parent_col_names.append(parent_col)

# ****************************************
## fill in missing parent info by borrowing owner info
owners_FULL_set = owners_df[owner_col_names].stack().dropna().unique().tolist() # from owners_df
owners_researched_set = list(set(owner_parent_links_df.index.to_list()))#+['Unknown'] # only existing owners, plus 'Unknown'
owners_diff = list(set(owners_FULL_set)-set(owners_researched_set))
owners_diff.append('Unknown')

# update owner_parent_links_df with these extra owners
owner_parent_links_df = pandas.concat([owner_parent_links_df, pandas.DataFrame(index=owners_diff, columns=owner_parent_links_df.columns)])
owner_parent_links_df['Parent1'].loc[owners_diff] = owners_diff
owner_parent_links_df['Parent1%'].loc[owners_diff] = '100.00%'

# ****************************************
# update parents_df with these as well
# note countries will be unknkown...
parents_set = list(set(parents_df.index.to_list()))
parents_diff = list(set(owners_diff)-set(parents_set))
parents_diff.append('Unknown')
parents_df = pandas.concat([parents_df, pandas.DataFrame(numpy.nan, index=parents_diff, columns=parents_df.columns)])
parents_df.loc[parents_diff,'ParentHQCountry'] = 'Unknown'

## replace "--" with NaN, removing empty rows

the dataset is structured to have -- wherever there's a lookup value that doesn't exist; replacing it with NaN (numpy.nan) allows pandas to treat it as a null value, which makes calculations much easier

In [169]:
# replace -- entries with NaN
pipes_df_orig = pipes_df_orig.replace('--', numpy.nan)
pipes_df_orig = pipes_df_orig[pipes_df_orig['PipelineName']!='']
pipes_df_orig = pipes_df_orig[pipes_df_orig['Wiki']!='']
country_ratios_df.replace('--', numpy.nan, inplace=True)

# km by country, km by region calculations

In [170]:
status_list = ['Proposed', 'Construction', 'Shelved', 'Cancelled', 'Operating', 'Idle', 'Mothballed', 'Retired']
country_list = sorted(list(set(country_ratios_df['Country'])))
region_list = sorted(list(set(country_ratios_df['Region'])))

In [171]:
country_ratios_gas_df = country_ratios_df[country_ratios_df['Fuel']=='Gas']

km_by_country_df = pandas.DataFrame(columns=status_list, index=country_list)
km_by_region_df = pandas.DataFrame(columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    country_ratios_gas_df_status = country_ratios_gas_df[country_ratios_gas_df['Status']==status]
    km_by_country_df[status] = country_ratios_gas_df_status.groupby('Country')['MergedKmByCountry'].sum()

print('===regional calculations===')
for status in status_list:
    print(status)
    country_ratios_gas_df_status = country_ratios_gas_df[country_ratios_gas_df['Status']==status]
    km_by_region_df[status] = country_ratios_gas_df_status.groupby('Region')['MergedKmByCountry'].sum()

# # fill NaN with 0.0
km_by_region_df = km_by_region_df.fillna(0)
km_by_country_df = km_by_country_df.fillna(0)

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===regional calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


In [172]:
km_by_country_df.index.to_list()

['Afghanistan',
 'Albania',
 'Algeria',
 'Angola',
 'Area of overlap Australia/Indonesia',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Belarus',
 'Belgium',
 'Benin',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Costa Rica',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 "Côte d'Ivoire",
 'DR Congo',
 'Denmark',
 'Disputed (Western Sahara/Mauritania)',
 'Djibouti',
 'Dominica',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Finland',
 'France',
 'Gabon',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guadeloupe',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Honduras',
 'Hong Kong',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran',
 'Iraq',
 'Ireland',
 'Israel',
 'Italy',
 'Japan',
 'Joint development area Australia/East

In [173]:
km_by_region_df['Proposed+Construction'] = km_by_region_df[['Proposed','Construction']].sum(axis=1)
km_by_region_df.sort_values(by='Proposed+Construction', inplace=True)
km_by_region_df = km_by_region_df[['Proposed', 'Construction', 'Proposed+Construction', 'Shelved', 'Cancelled', 'Operating', 'Idle', 'Mothballed', 'Retired']]
km_by_region_df.sort_values('Proposed+Construction', ascending=False, inplace=True)

In [174]:
km_by_country_df['Proposed+Construction'] = km_by_country_df[['Proposed','Construction']].sum(axis=1)
km_by_country_df.sort_values(by='Proposed+Construction', inplace=True)
km_by_country_df = km_by_country_df[['Proposed', 'Construction', 'Proposed+Construction', 'Shelved', 'Cancelled', 'Operating', 'Idle', 'Mothballed', 'Retired']]
km_by_country_df.sort_values('Proposed+Construction', ascending=False, inplace=True)

# projects by country, by region

In [175]:
country_ratios_gas_df = country_ratios_df[country_ratios_df['Fuel']=='Gas']

num_by_country_df = pandas.DataFrame(columns=status_list, index=country_list)
num_by_region_df = pandas.DataFrame(columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    country_ratios_gas_df_status = country_ratios_gas_df[country_ratios_gas_df['Status']==status]
    num_by_country_df[status] = country_ratios_gas_df_status.groupby('ProjectID')['MergedKmByCountry'].sum()

print('===regional calculations===')
for status in status_list:
    print(status)
    country_ratios_gas_df_status = country_ratios_gas_df[country_ratios_gas_df['Status']==status]
    num_by_region_df[status] = country_ratios_gas_df_status.groupby('Region')['MergedKmByCountry'].sum()

# # fill NaN with 0.0
num_by_region_df = num_by_region_df.fillna(0)
num_by_country_df = num_by_country_df.fillna(0)

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===regional calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


# cost estimates (pipeline cost per km)

throwing out any estimates larger than USD 10 million per km for taking regional estimates

In [176]:
outliers_projectids = ['P2509','P0173','P1297', 'P4033', 'P0174','P3188','P2551', 'P4069', 'P0331', 'P1382','P0477']
# for gas...

In [177]:
# pull out only pipelines that have a KNOWN length AND a cost
country_ratios_with_length_and_cost_df = country_ratios_df.copy()[(country_ratios_df['Fuel']=='Gas') & 
                                    (country_ratios_df['CostUSDPerKm'].notna()) & 
                                    (country_ratios_df['LengthKnownKmByCountry'].notna()) &
                                    (country_ratios_df['LengthKnownKm']!=0) &
                                    #(country_ratios_df['CostUSDPerKm']<10e6) &
                                    ~(country_ratios_df['ProjectID'].isin(outliers_projectids))
                                   ]
#country_ratios_with_length_and_cost_df = country_ratios_with_length_and_cost_df[~country_ratios_with_length_and_cost_df.ProjectID.isin(outliers_projectids)]

### global mean value

In [178]:
country_ratios_with_length_and_cost_df['CostUSDPerKm'].drop_duplicates().mean()

3099032.3178019323

### calculate regional costs

In [179]:
pipes_costs_region_df = pandas.DataFrame(0, index=region_list, columns=['CostUSDPerKm','NumberOfCosts','NumberOfLengths'])

for region in region_list:
    print(region)
    country_ratios_region_df = country_ratios_with_length_and_cost_df[country_ratios_with_length_and_cost_df['Region']==region]
    pipes_costs_region_df.loc[region]['CostUSDPerKm'] = country_ratios_region_df['CostUSDPerKm'].mean()
    pipes_costs_region_df.loc[region]['NumberOfCosts'] = list(set(country_ratios_region_df['ProjectID'])).__len__()
    pipes_costs_region_df.loc[region]['NumberOfLengths'] = list(set(country_ratios_region_df['ProjectID'])).__len__()
    

Australia and New Zealand
East Asia
Eurasia
Europe
Latin America and the Caribbean
Middle East and North Africa
North America
SE Asia
South Asia
Sub-Saharan Africa


### table for methodology with regional cost info

In [180]:
pipes_costs_region_df

,CostUSDPerKm,NumberOfCosts,NumberOfLengths
Australia and New Zealand,1306846,10,10
East Asia,1701998,55,55
Eurasia,4115383,46,46
Europe,3129315,111,111
Latin America and the Caribbean,3502921,24,24
Middle East and North Africa,2809259,31,31
North America,4723158,140,140
SE Asia,2366359,8,8
South Asia,1663439,17,17
Sub-Saharan Africa,4001946,5,5


### calculate country-level costs

In [181]:
#country_list_for_costs = sorted(list(set(country_ratios_with_length_and_cost_df['Country'])))
pipes_costs_country_df = pandas.DataFrame(0, index=country_list, columns=['CostUSDPerKm','NumberOfCosts','NumberOfLengths'])

for country in country_list:#_for_costs:
    #print(country)
    country_ratios_country_df = country_ratios_with_length_and_cost_df[country_ratios_with_length_and_cost_df['Country']==country]
    pipes_costs_country_df.loc[country,'CostUSDPerKm'] = country_ratios_country_df['CostUSDPerKm'].mean()
    pipes_costs_country_df.loc[country,'NumberOfCosts'] = list(set(country_ratios_country_df['ProjectID'])).__len__()
    pipes_costs_country_df.loc[country,'NumberOfLengths'] = list(set(country_ratios_country_df['ProjectID'])).__len__()

# tables etc.

## table for regional totals

In [182]:
if 'Total' in km_by_region_df.index:
    km_by_region_df.drop(index='Total', inplace=True)   
    total = km_by_region_df.sum(axis=0)
    total.name='Total'
    km_by_region_df = km_by_region_df.append(total)

else:
    total = km_by_region_df.sum(axis=0)
    total.name='Total'
    km_by_region_df = km_by_region_df.append(total)

km_by_region_df

,Proposed,Construction,Proposed+Construction,Shelved,Cancelled,Operating,Idle,Mothballed,Retired
East Asia,"31,380.90000","26,377.70000","57,758.60000","2,065.71000",475.27000,"92,288.09000",0.00000,0.00000,41.00000
Eurasia,"18,321.40000","5,153.38000","23,474.78000",272.16000,"6,647.69000","128,934.76000",0.00000,"3,300.05000","1,784.94000"
South Asia,"4,221.03000","17,140.20000","21,361.23000","6,357.71000","6,143.81000","29,669.32000",0.00000,0.00000,35.00000
Sub-Saharan Africa,"18,823.08000","1,582.00000","20,405.08000","1,678.15000",176.96000,"7,031.52000",0.00000,0.00000,0.00000
Europe,"14,399.21000","2,836.20000","17,235.41000","5,705.03000","14,014.71000","110,152.27000",0.00000,"4,734.21000",821.36000
North America,"11,837.51000","2,768.33000","14,605.84000","10,916.42000","13,243.21000","420,872.56000",331.00000,0.00000,0.00000
Latin America and the Caribbean,"12,772.26000","1,304.00000","14,076.26000","2,187.28000","1,301.38000","39,241.07000",0.00000,224.00000,0.00000
Australia and New Zealand,"12,563.28000",580.00000,"13,143.28000",488.23000,"6,194.69000","31,025.88000",0.00000,0.00000,0.00000
Middle East and North Africa,"5,269.15000","6,726.91000","11,996.06000","5,963.24000","4,144.74000","44,502.38000",0.00000,"1,106.60000",39.28000
SE Asia,"9,222.66000",984.30000,"10,206.96000","2,634.21000","1,423.81000","16,879.66000",0.00000,0.00000,0.00000


## table for km by country

In [183]:
# saves all countries to an Excel file for reference

km_by_country_df.sort_values('Proposed+Construction', ascending=False)

if 'Total' in km_by_country_df.index:
    km_by_country_df.drop(index='Total', inplace=True)   
    total = km_by_country_df.sum(axis=0)
    total.name='Total'
    km_by_country_df = km_by_country_df.append(total)

else:
    total = km_by_country_df.sum(axis=0)
    total.name='Total'
    km_by_country_df = km_by_country_df.append(total)

#km_by_country_df.to_excel('GGIT-km-by-country.xlsx')

# table for stranded asset calculations

## country-level capex estimates

In [184]:
country_ratios_gas_df

,PipelineName,SegmentName,ProjectID,Country,LengthEstimateKmByCountry,LengthEstimateKm,LengthPerCountryFraction,Owner,StartYearEarliest,Region,...,CostUSD,CostEuro,LengthKnownKm,LengthKnownKmByCountry,MergedKmByPipeline,CostUSDPerKm,CostEuroPerKm,MergedKmByCountry,Owner,Parent
0,Turkmenistan-Afghanistan-Pakistan-India Gas Pi...,,P0766,Afghanistan,"1,033.68260","1,814.00000",0.50000,Turkmengaz (100%),NaN,Middle East and North Africa,...,"10,000,000,000.00000","8,771,929,824.56000","1,814.00000",907.05000,"1,814.00000","5,512,679.16000","4,835,683.48000",907.05000,Turkmengaz [100.00%],Turkmengaz [100.00%]
2,Sheberghan-Mazar-i-Sharif Gas Pipeline,,P3922,Afghanistan,129.65016,94.50000,1.00000,NaN,NaN,Middle East and North Africa,...,NaN,NaN,94.50000,94.50000,94.50000,NaN,NaN,94.50000,Unknown [unknown %],Unknown [unknown %]
4,Ionian Adriatic Gas Pipeline,,P0702,Albania,165.02915,540.00000,0.33000,Albgaz Sha; BH-Gas d.o.o.; Plinacro; Montenegr...,2025,Europe,...,"668,040,000.00000","586,000,000.00000",540.00000,178.31000,540.00000,"1,237,111.11000","1,085,185.19000",178.31000,Albgaz Sha [unknown %]; BH-Gas d.o.o. [unknown...,Albgaz Sha [unknown %]; BH-Gas d.o.o. [unknown...
5,Trans-Adriatic Gas Pipeline,,P0724,Albania,215.35431,878.00000,0.29000,Trans Adriatic Pipeline AG (100%),2020,Europe,...,"4,500,000,000.00000","3,947,368,421.05000",878.00000,251.27000,878.00000,"5,125,284.74000","4,495,863.81000",251.27000,Trans Adriatic Pipeline AG [600.00%],BP [20.00%]; SOCAR [20.00%]; Snam [20.00%]; Fl...
6,South Stream Gas Pipeline,,P0760,Albania,18.31370,"2,380.00000",0.01000,South Stream B.V.,NaN,Europe,...,"15,500,000,000.00000","13,596,491,228.07000","2,380.00000",12.03000,"2,380.00000","6,512,605.04000","5,712,811.44000",12.03000,South Stream B.V. [unknown %],Agence des participations de l'État [unknown %...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4377,FSRU to Thai Binh Gas Pipeline,,P3151,Vietnam,50,50.00000,1.00000,NaN,2026,SE Asia,...,NaN,NaN,50.00000,50.00000,50.00000,NaN,NaN,50.00000,Unknown [unknown %],Unknown [unknown %]
4378,East to Southwest Gas Pipeline,,P3152,Vietnam,,,1.00000,NaN,2026,SE Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown [unknown %],Unknown [unknown %]
4379,Trans-ASEAN Gas Pipeline,,P3153,Vietnam,,,1.00000,NaN,2026,SE Asia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown [unknown %],Unknown [unknown %]
4380,Puerto Rico-Virgin Islands Pipeline,,P1253,Virgin Islands (U.S.),27.24702,,0.66000,NaN,NaN,Latin America and the Caribbean,...,NaN,NaN,NaN,NaN,41.44000,NaN,NaN,27.25000,Unknown [unknown %],Unknown [unknown %]


In [185]:
country_ratios_gas_df = country_ratios_df.loc[country_ratios_df['Fuel']=='Gas']
country_ratios_gas_df = country_ratios_gas_df.reset_index(drop=True)
country_ratios_gas_df.loc[:,'CostUSDEstimate'] = numpy.nan

for idx,row in country_ratios_gas_df.iterrows():
    # calculate cost
    cntry = row.Country
    region = row.Region
    km_by_cntry = row.LengthKnownKmByCountry
    #print(row.PipelineName)
    country_ratios_gas_df.loc[idx,'CostUSDEstimate'] = pipes_costs_region_df.loc[region, 'CostUSDPerKm'] * km_by_cntry

# replace any known costs now

# known_costs = country_ratios_gas_df.loc[(~country_ratios_gas_df.LengthKnownKmByCountry.isna())&
#                       (~country_ratios_gas_df.CostUSDPerKm.isna()),'LengthKnownKmByCountry'] * \
# country_ratios_gas_df.loc[(~country_ratios_gas_df.LengthKnownKmByCountry.isna())&
#                       (~country_ratios_gas_df.CostUSDPerKm.isna()),'CostUSDPerKm']

# country_ratios_gas_df.loc[(~country_ratios_gas_df.LengthKnownKmByCountry.isna())&
#                       (~country_ratios_gas_df.CostUSDPerKm.isna()),'CostUSDEstimate'] = known_costs

country_ratios_gas_df.loc[(~country_ratios_gas_df.LengthKnownKmByCountry.isna())&
                      (~country_ratios_gas_df.CostUSDPerKm.isna()),'CostUSDEstimate'] = \
country_ratios_gas_df.loc[(~country_ratios_gas_df.LengthKnownKmByCountry.isna())&
                      (~country_ratios_gas_df.CostUSDPerKm.isna()),'LengthKnownKmByCountry'] * \
country_ratios_gas_df.loc[(~country_ratios_gas_df.LengthKnownKmByCountry.isna())&
                      (~country_ratios_gas_df.CostUSDPerKm.isna()),'CostUSDPerKm']

country_sums_USDBillion_df = pandas.DataFrame(country_ratios_gas_df.loc[(country_ratios_gas_df.Status.isin(['Construction','Proposed']))].groupby('Country')['CostUSDEstimate'].sum(min_count=1)/1e9)

In [186]:
capex_by_country_df = pandas.DataFrame(columns=status_list, index=country_list)
capex_by_region_df = pandas.DataFrame(columns=status_list, index=region_list)

print('===country-level calculations===')
for status in status_list:
    print(status)
    country_ratios_gas_df_status = country_ratios_gas_df[country_ratios_gas_df.Status==status]
    capex_by_country_df[status] = country_ratios_gas_df_status.groupby('Country')['CostUSDEstimate'].sum()/1e9

print('===regional calculations===')
for status in status_list:
    print(status)
    country_ratios_gas_df_status = country_ratios_gas_df[country_ratios_gas_df.Status==status]
    capex_by_region_df[status] = country_ratios_gas_df_status.groupby('Region')['CostUSDEstimate'].sum()/1e9

# # fill NaN with 0.0
capex_by_region_df = capex_by_region_df.fillna(0)
capex_by_country_df = capex_by_country_df.fillna(0)

===country-level calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired
===regional calculations===
Proposed
Construction
Shelved
Cancelled
Operating
Idle
Mothballed
Retired


In [187]:
capex_by_region_df['Proposed+Construction'] = capex_by_region_df[['Proposed','Construction']].sum(axis=1)
#capex_by_region_df.sort_values(by='Proposed+Construction', inplace=True)
capex_by_region_df = capex_by_region_df[['Proposed', 'Construction', 'Proposed+Construction', 'Shelved', 'Cancelled', 'Operating', 'Idle', 'Mothballed', 'Retired']]
capex_by_region_df.sort_values('Proposed+Construction', ascending=False, inplace=True)

In [188]:
capex_by_country_df['Proposed+Construction'] = capex_by_country_df[['Proposed','Construction']].sum(axis=1)
#capex_by_country_df.sort_values(by='Proposed+Construction', inplace=True)
capex_by_country_df = capex_by_country_df[['Proposed', 'Construction', 'Proposed+Construction', 'Shelved', 'Cancelled', 'Operating', 'Idle', 'Mothballed', 'Retired']]
capex_by_country_df.sort_values('Proposed+Construction', ascending=False, inplace=True)

In [189]:
capex_by_region_df

,Proposed,Construction,Proposed+Construction,Shelved,Cancelled,Operating,Idle,Mothballed,Retired
East Asia,46.64767,38.74089,85.38856,2.28285,2.12287,154.59253,0.00000,0.00000,0.06978
North America,64.69183,13.88105,78.57288,56.41560,87.62897,"1,869.57470",1.56337,0.00000,0.00000
Eurasia,53.94520,20.94491,74.89011,1.69141,19.61631,541.17621,0.00000,13.58097,4.13559
Sub-Saharan Africa,58.43414,4.01855,62.45269,0.38819,0.00000,28.13976,0.00000,0.00000,0.00000
Europe,36.31740,5.91164,42.22904,16.74955,46.00583,277.63773,0.00000,14.23363,2.57029
Middle East and North Africa,19.26246,20.81695,40.07941,16.59612,5.48053,120.41321,0.00000,3.10873,0.00000
Latin America and the Caribbean,33.20036,3.37580,36.57616,9.93421,4.41344,137.45837,0.00000,0.78465,0.00000
South Asia,7.44634,26.09272,33.53906,2.01850,9.89798,46.39780,0.00000,0.00000,0.05822
SE Asia,19.21205,2.73709,21.94914,2.33740,3.36925,39.60866,0.00000,0.00000,0.00000
Australia and New Zealand,13.78342,0.75797,14.54139,0.35704,6.96150,39.70444,0.00000,0.00000,0.00000


In [190]:
capex_by_region_df

,Proposed,Construction,Proposed+Construction,Shelved,Cancelled,Operating,Idle,Mothballed,Retired
East Asia,46.64767,38.74089,85.38856,2.28285,2.12287,154.59253,0.00000,0.00000,0.06978
North America,64.69183,13.88105,78.57288,56.41560,87.62897,"1,869.57470",1.56337,0.00000,0.00000
Eurasia,53.94520,20.94491,74.89011,1.69141,19.61631,541.17621,0.00000,13.58097,4.13559
Sub-Saharan Africa,58.43414,4.01855,62.45269,0.38819,0.00000,28.13976,0.00000,0.00000,0.00000
Europe,36.31740,5.91164,42.22904,16.74955,46.00583,277.63773,0.00000,14.23363,2.57029
Middle East and North Africa,19.26246,20.81695,40.07941,16.59612,5.48053,120.41321,0.00000,3.10873,0.00000
Latin America and the Caribbean,33.20036,3.37580,36.57616,9.93421,4.41344,137.45837,0.00000,0.78465,0.00000
South Asia,7.44634,26.09272,33.53906,2.01850,9.89798,46.39780,0.00000,0.00000,0.05822
SE Asia,19.21205,2.73709,21.94914,2.33740,3.36925,39.60866,0.00000,0.00000,0.00000
Australia and New Zealand,13.78342,0.75797,14.54139,0.35704,6.96150,39.70444,0.00000,0.00000,0.00000


In [191]:
total

Proposed                138,810.48000
Construction             65,453.02000
Proposed+Construction   204,263.50000
Shelved                  38,268.14000
Cancelled                53,766.27000
Operating               920,597.51000
Idle                        331.00000
Mothballed                9,364.86000
Retired                   2,721.58000
Name: Total, dtype: float64

In [192]:
if 'Total' in capex_by_region_df.index:
    capex_by_region_df.drop(index='Total', inplace=True)   
    total = capex_by_region_df.sum(axis=0)
    #total.name='Total'
    capex_by_region_df.loc['Total'] = total

else:
    total = capex_by_region_df.sum(axis=0)
    #total.name='Total'
    capex_by_region_df.loc['Total'] = total

In [193]:
if 'Total' in capex_by_country_df.index:
    capex_by_country_df.drop(index='Total', inplace=True)   
    total = capex_by_country_df.sum(axis=0)
    #total.name='Total'
    capex_by_country_df.loc['Total'] = total

else:
    total = capex_by_country_df.sum(axis=0)
    #total.name='Total'
    capex_by_country_df.loc['Total'] = total

## print out country-level stranded assets for Jenny's report

In [194]:
country_list_gas_plants_report = ['Canada',
                                  'United States']

pandas.options.display.float_format = '{:,.5f}'.format
capex_by_country_df.loc[country_list_gas_plants_report,['Proposed','Construction']].reindex(country_list_gas_plants_report)

,Proposed,Construction
Canada,4.17408,5.72405
United States,50.87955,4.81773


In [195]:
km_by_country_df.loc[country_list_gas_plants_report,['Proposed','Construction','Operating']].reindex(country_list_gas_plants_report)

,Proposed,Construction,Operating
Canada,685.54000,"1,074.84000","61,253.40000"
United States,"8,777.40000",986.49000,"342,197.93000"


In [196]:
iom = pandas.Series(numpy.zeros(capex_by_country_df.columns.__len__()))#,dtype='float64')
iom.index = capex_by_country_df.columns
iom.name='Macao'
if 'Macao' not in capex_by_country_df.index:
    capex_by_country_df = capex_by_country_df.append(iom)

if 'Macao' not in km_by_country_df.index:
    km_by_country_df = km_by_country_df.append(iom)

In [197]:
country_list_gas_plants_report = ['China',
                                  'South Korea',
                                  'Japan',
                                  'Taiwan',
                                  'Macao',
                                  'Hong Kong']

pandas.options.display.float_format = '{:,.5f}'.format
capex_by_country_df.loc[country_list_gas_plants_report,['Proposed','Construction']].reindex(country_list_gas_plants_report)

,Proposed,Construction
China,44.09012,38.52779
South Korea,0.00000,0.00000
Japan,0.08067,0.17600
Taiwan,1.49095,0.03710
Macao,0.00000,0.00000
Hong Kong,0.00000,0.00000


In [198]:
km_by_country_df.loc[country_list_gas_plants_report,['Proposed','Construction','Operating']].reindex(country_list_gas_plants_report)

,Proposed,Construction,Operating
China,"29,806.42000","26,325.90000","87,076.84000"
South Korea,0.00000,0.00000,0.00000
Japan,47.40000,30.00000,"4,307.01000"
Taiwan,876.00000,21.80000,853.00000
Macao,0.00000,0.00000,0.00000
Hong Kong,0.00000,0.00000,51.24000


In [199]:
iom = pandas.Series(numpy.zeros(capex_by_country_df.columns.__len__()))#,dtype='float64')
iom.index = capex_by_country_df.columns
iom.name='Isle of Man'
if 'Isle of Man' not in capex_by_country_df.index:
    capex_by_country_df = capex_by_country_df.append(iom)

if 'Isle of Man' not in km_by_country_df.index:
    km_by_country_df = km_by_country_df.append(iom)

In [200]:
country_list_gas_plants_report = ['Austria',
                                'Belgium',
                                'Bulgaria',
                                'Croatia',
                                'Cyprus',
                                'Czech Republic',
                                'Denmark',
                                'Estonia',
                                'Finland',
                                'France',
                                'Germany',
                                'Greece',
                                'Hungary',
                                'Ireland',
                                'Italy',
                                'Latvia',
                                'Lithuania',
                                'Malta',
                                'Netherlands',
                                'Poland',
                                'Portugal',
                                'Romania',
                                'Slovakia',
                                'Slovenia',
                                'Spain',
                                'Sweden',
                                'Albania',
                                'Belarus',
                                'Bosnia and Herzegovina',
                                'Isle of Man',
                                'Moldova',
                                'Montenegro',
                                'North Macedonia',
                                'Norway',
                                'Serbia',
                                'Switzerland',
                                'Turkey',
                                'Ukraine',
                                'United Kingdom']

pandas.options.display.float_format = '{:,.5f}'.format
capex_by_country_df.loc[country_list_gas_plants_report,['Proposed','Construction']].reindex(country_list_gas_plants_report)

,Proposed,Construction
Austria,0.18710,0.00000
Belgium,0.00000,0.00000
Bulgaria,1.25791,0.31963
Croatia,1.08843,0.00000
Cyprus,2.77088,0.00000
Czech Republic,0.00692,0.00000
Denmark,0.00000,1.65720
Estonia,0.00000,0.00376
Finland,0.00000,0.00000
France,2.21929,0.00000


In [201]:
km_by_country_df.loc[country_list_gas_plants_report,['Proposed','Construction','Operating']].reindex(country_list_gas_plants_report)

,Proposed,Construction,Operating
Austria,59.79000,0.00000,831.03000
Belgium,0.00000,0.00000,"1,406.52000"
Bulgaria,488.34000,346.26000,"4,543.09000"
Croatia,858.44000,0.00000,744.76000
Cyprus,911.55000,0.00000,0.00000
Czech Republic,2.21000,0.00000,"1,111.89000"
Denmark,0.00000,423.99000,907.81000
Estonia,0.00000,1.20000,653.16000
Finland,0.00000,0.00000,"1,118.15000"
France,749.76000,0.00000,"8,131.19000"
